Here is a demonstration of how to use the tools in this repo to construct syndrome extraction circuits for the rotated surface code and to evaluate a threshold for logical state preparation under a simple single-qubit error model.

In [2]:
import sys
import os
sys.path.append(os.path.abspath('..'))

In [3]:
import pymatching
import numpy as np

from scipy import sparse
from stim import Circuit
from csscode.cssCode import cssCode

from codes.code_tools import commutation_test
from codes.rotated_surface_code_coordinates import rsurf_stabilizer_generators as stabgens

Functions for initializing the rotated surface code data:

In [ ]:
def rsurf_code(L:int):
    sx,sz = stabgens(L)
    return cssCode(sx,sz)

def zlogical(L:int):
    assert L%2==1
    return set([q for q in range(L)])

Test that the $X$-generators of the code commute with the logical-$Z$ operator using the commutation_test function:

In [ ]:
for L in [7,11,15]:
    print(commutation_test([zlogical(L)],stabgens(L)[0]))

Circuits for syndrome extraction and logical Z measurements of the rotated surface code:

In [ ]:
def z_syn_circ(circuit:Circuit,code:cssCode):
    for zcheck in code.check_dict[True]:
        anc_qub = code.Nqubits+zcheck
        for dat_qub in code.check_dict[True][zcheck]:
            circuit.append("CX",[dat_qub,anc_qub])
        circuit.append("MR",anc_qub)

def x_syn_circ(circuit:Circuit,code:cssCode):
    for xcheck in code.check_dict[False]:
        anc_qub = code.Nqubits+len(code.check_dict[True])+xcheck
        circuit.append("H",anc_qub)
        for dat_qub in code.check_dict[False][xcheck]:
            circuit.append("CX",[anc_qub,dat_qub])
        circuit.append("H",anc_qub)
        circuit.append("MR",anc_qub)

def zL_meas_circ(circuit:Circuit,code:cssCode):
    L = int(np.sqrt(code.Nqubits)) ## Only intended for rotated surface code
    zL_qub = code.Nqubits+len(code.check_dict[True])+len(code.check_dict[False])
    for dat_qub in zlogical(L):
        circuit.append("CX",[dat_qub,zL_qub])
    circuit.append("MR",zL_qub)

In [ ]:
def apply_single_qubit_noise(circuit:Circuit,code:cssCode,p:list[float]):
    circuit.append("PAULI_CHANNEL_1",list(code.qubits),[p[0],p[1],p[2]])

One round of single-qubit noise, followed by perfect syndrome extraction and then a logical measurement:

In [ ]:
rs3 = rsurf_code(3)

circ = Circuit()
apply_single_qubit_noise(circ,rs3,[0.1,0.0,0.1])
z_syn_circ(circ,rs3)
x_syn_circ(circ,rs3)
zL_meas_circ(circ,rs3)
circ.diagram()

Parity check matrices for the rotated surface code:

In [ ]:
zpcm = rs3.to_check_matrices()[True]
zrow = [elem[0] for elem in zpcm]
zcol = [elem[1] for elem in zpcm]
zdata = [elem[2] for elem in zpcm]

hz = sparse.coo_array((zdata,(zrow,zcol))).toarray()

xpcm = rs3.to_check_matrices()[False]
xrow = [elem[0] for elem in xpcm]
xcol = [elem[1] for elem in xpcm]
xdata = [elem[2] for elem in xpcm]

hx = sparse.coo_array((xdata,(xrow,xcol))).toarray()

print('hz = ' +'\n'+str(hz))
print()
print('hx = ' +'\n'+str(hx))

In [ ]:
num_shots = 10000

py = 0
pz = 0

for px in [0.07, 0.10, 0.13]:

    print('px = ' + str(px))

    for L in [11,15,19,23]:
        num_errs = 0
        rs = rsurf_code(L)

        zpcm = rs.to_check_matrices()[True]
        zrow = [elem[0] for elem in zpcm]
        zcol = [elem[1] for elem in zpcm]
        zdata = [elem[2] for elem in zpcm]

        hz = sparse.coo_array((zdata,(zrow,zcol))).toarray()
        zL = np.array([1]*L+[0]*(rs.Nqubits-L),dtype = int)

        matching = pymatching.Matching(hz)

        ## Initialize the state (all qubits initially in |0> state)
        ## Apply single qubit noise
        ## Perform syndrome extraction
        ## Measure the logical-Z operator

        circ = Circuit()
        apply_single_qubit_noise(circ,rs,[px,py,pz])
        z_syn_circ(circ,rs)
        x_syn_circ(circ,rs)
        zL_meas_circ(circ,rs)


        ## Generate a sampler of the circuit then perform num_shots trials
        sampler = circ.compile_sampler()
        results = sampler.sample(shots=num_shots)

        ## Assess each result for a logical failure
        for res in results:

            ## Extract the Z portion of the syndrome (the first measurement outcomes from each trial)
            ## Also extract the observed value of the logical-Z operator (the last measurement outcome from each trial)
            zsynd = res[:len(rs.code[True])]
            zmeas = res[-1]

            ## Obtain the predicted error from the decoder for the observed Z-syndrome
            prediction = matching.decode(zsynd)

            ## Perform error-recovery by modifying the observed logical-Z output
            ## The dot product of the Z-logical operator and the predicted X-error tells us if we need to flip zmeas
            ## The initial value of zL is |0>, so if the correction predicts |1>, then error has occurred
            num_errs += (zmeas + zL@prediction.T)%2

        print('L = ' + str(L) + ', num errs = ' + str(num_errs))
    
    print()


Below are circuits for a phenomenological noise model. In particular, we assume the extraction circuit to be perfect, except with a probability $p_{meas}$ of flipping the reported outcome.

In [5]:
def phenom_z_syn_circ(circuit:Circuit,code:cssCode,pmeas:float):
    for zcheck in code.check_dict[True]:
        anc_qub = code.Nqubits+zcheck
        
        for dat_qub in code.check_dict[True][zcheck]:
            circuit.append("CX",[dat_qub,anc_qub])
        
        circuit.append("X_ERROR",anc_qub,pmeas)
        circuit.append("MR",anc_qub)

def phenom_x_syn_circ(circuit:Circuit,code:cssCode,pmeas:float):
    for xcheck in code.check_dict[False]:
        anc_qub = code.Nqubits+len(code.check_dict[True])+xcheck

        circuit.append("H",anc_qub)

        for dat_qub in code.check_dict[False][xcheck]:
            circuit.append("CX",[anc_qub,dat_qub])

        circuit.append("H",anc_qub)
        circuit.append("X_ERROR",anc_qub,pmeas)        
        circuit.append("MR",anc_qub)

def phenom_syndrome_circuit(L:int,p1:float,pmeas:float):

    rs = rsurf_code(L)
    circ = Circuit()
    for round in range(L):
        apply_single_qubit_noise(circ,rs,[p1,p1,p1])
        phenom_z_syn_circ(circ,rs,pmeas)
        phenom_x_syn_circ(circ,rs,pmeas)
    zL_meas_circ(circ,rs)

    return circ

We can now sample the circuit to establish an error threshold for the identity operation of the rotated surface code.

In [ ]:
num_shots = 200
num_errs = 0



Below are the noisy versions of the above syndrome measurement circuits, where the model is single-qubit and two-qubit depolarizing noise with error probabilities of $p1$ and $p2$, respectively.

In each circuit noisy state preparation and measurement is modeled with single-qubit depolarizing noise following preparation, or preceding measurement, respectively.

Each noisy two-qubit gate is modeled as the ideal gate followed by a two-qubit depolarizing channel.

In [ ]:
def noisy_z_syn_circ(circuit:Circuit,code:cssCode,p1:float,p2:float):
    for zcheck in code.check_dict[True]:
        anc_qub = code.Nqubits+zcheck
        
        circuit.append("DEPOLARIZE1",anc_qub,p1)
        
        for dat_qub in code.check_dict[True][zcheck]:
            circuit.append("CX",[dat_qub,anc_qub])
            circuit.append("DEPOLARIZE2",[dat_qub,anc_qub],p2)
        
        circuit.append("DEPOLARIZE1",anc_qub,p1)
        circuit.append("MR",anc_qub)

def noisy_x_syn_circ(circuit:Circuit,code:cssCode,p1:float,p2:float):
    for xcheck in code.check_dict[False]:
        anc_qub = code.Nqubits+len(code.check_dict[True])+xcheck

        circuit.append("H",anc_qub)
        circuit.append("DEPOLARIZE1",anc_qub,p1)

        for dat_qub in code.check_dict[False][xcheck]:
            circuit.append("CX",[anc_qub,dat_qub])
            circuit.append("DEPOLARIZE2",[anc_qub,dat_qub],p2)
        
        circuit.append("H",anc_qub)
        circuit.append("MR",anc_qub)

def noisy_syndrome_circuit(L:int,p1,p2):

    rs = rsurf_code(L)
    circ = Circuit()
    for round in range(L):
        apply_single_qubit_noise(circ,rs,[p1,p1,p1])
        noisy_z_syn_circ(circ,rs,p1,p2)
        noisy_x_syn_circ(circ,rs,p1,p2)
    zL_meas_circ(circ,rs)

    return circ

Below we'll run a numerical experiment to estimate the threshold for the toric code under this noise model.
In particular, we will:
1. Prepare a state in a simultaneous $\bar{X}_A \bar{X}_B$, $\bar{Z}_A \bar{Z}_B$ eigenstate of the $L \times L$ toric code
2. Perform $L$ rounds of noisy syndrome extraction
3. Decode both the $X$ and $Z$ check systems
4. Readout the $\bar{X}_A \bar{X}_B$, $\bar{Z}_A \bar{Z}_B$ eigenvalues (with a noiseless circuit)
5. Compare the decoded output eigenvalues with the initialized eigenvalues to see if a logical error has occurred.